# Resource estimates for 18 O tapered circuits

In [1]:
import pennylane as qml
from pennylane import numpy as np

import ionizer
from ionizer.transforms import ionize, commute_through_ms_gates, single_qubit_fusion_gpi

In [2]:
dev = qml.device("default.qubit", wires=range(5))

@qml.qfunc_transform
def expand_rot_and_remove_zeros(tape):
    for op in tape:
        if op.name == "Rot":
            if not np.isclose(op.data[0], 0.0):
                qml.RZ(op.data[0], wires=op.wires)
            if not np.isclose(op.data[1], 0.0):
                qml.RY(op.data[1], wires=op.wires)
            if not np.isclose(op.data[2], 0.0):
                qml.RZ(op.data[2], wires=op.wires)
        else:
            qml.apply(op)

@qml.qnode(dev)
@expand_rot_and_remove_zeros
@qml.transforms.single_qubit_fusion()
def tapered_circuit_simplified(params):
    qml.RY(params[0], wires=dev.wires[0])
    for idx in range(len(dev.wires)-1):
        qml.SingleExcitation.compute_decomposition(params[idx+1], wires=[dev.wires[idx], dev.wires[idx+1]])
    return qml.probs()

In [3]:
params = np.array([ 1.20436673, -1.13296563, -0.16546356, -0.06453663,  1.0023775 ,
         1.29885146,  0.12871688,  0.0831842 ], requires_grad=True)

In [4]:
print(qml.draw(tapered_circuit_simplified)(params))

0: ──RZ(1.91)──RY(0.85)──RZ(2.64)─╭X──RZ(0.57)──╭X──RZ(1.57)──RY(1.57)──RZ(0.79)──────────────
1: ──RZ(0.79)──RY(1.57)───────────╰●──RY(-0.57)─╰●──RZ(1.57)──RY(1.57)─╭X─────────RZ(0.08)──╭X
2: ──RZ(0.79)──RY(1.57)────────────────────────────────────────────────╰●─────────RY(-0.08)─╰●
3: ──RZ(0.79)──RY(1.57)───────────────────────────────────────────────────────────────────────
4: ──RZ(0.79)──RY(1.57)───────────────────────────────────────────────────────────────────────

───────────────────────────────────────────────────────────────────────────────────────────────────
───RZ(1.57)──RY(1.57)──RZ(0.79)────────────────────────────────────────────────────────────────────
───RZ(1.57)──RY(1.57)─╭X─────────RZ(0.03)──╭X──RZ(1.57)──RY(1.57)──RZ(0.79)────────────────────────
──────────────────────╰●─────────RY(-0.03)─╰●──RZ(1.57)──RY(1.57)─╭X─────────RZ(-0.50)─╭X──RZ(1.57)
──────────────────────────────────────────────────────────────────╰●─────────RY(0.50)──╰●──RZ(3.14)

──────────────────────┤

## Transpile to Qiskit

In [5]:
from qiskit import QuantumCircuit
from qiskit import transpile

original_tape = tapered_circuit_simplified.qtape
original_qasm = original_tape.to_openqasm()

In [6]:
qiskit_circuit = QuantumCircuit.from_qasm_str(original_qasm)

transpiled_circuit = transpile(
    qiskit_circuit, 
    basis_gates=['rz', 'ry', 'cx', 'measure'],
    optimization_level=3
)                                 

In [7]:
# Convert back to a PennyLane function
new_qfunc = qml.from_qiskit(transpiled_circuit)

@qml.qnode(dev)
def tapered_circuit_transpiled():
    new_qfunc()
    # The Qiskit transpiler eats terminal RZs; we actually need
    # them when we measure the expval of a Hamiltonian, so add them back 
    qml.RZ(np.pi/4, wires=0)
    qml.RZ(np.pi/4, wires=1)
    qml.RZ(np.pi/4, wires=2)
    qml.RZ(np.pi/4, wires=3)
    qml.RZ(3 * np.pi/4, wires=4)
    return qml.probs()

In [8]:
qml.specs(tapered_circuit_transpiled)()

/home/olivia/.conda/envs/ionizer/lib/python3.11/site-packages/pennylane_qiskit/converter.py:227: UserWarning: pennylane_qiskit.converter: The Measure instruction is not supported by PennyLane, and has not been added to the template.
  warnings.warn(


{'resources': Resources(num_wires=5, num_gates=48, gate_types=defaultdict(<class 'int'>, {'RZ': 23, 'RY': 17, 'CNOT': 8}), gate_sizes=defaultdict(<class 'int'>, {1: 40, 2: 8}), depth=24, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 40, 2: 8}),
 'gate_types': defaultdict(int, {'RZ': 23, 'RY': 17, 'CNOT': 8}),
 'num_operations': 48,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 5,
 'num_trainable_params': 0,
 'depth': 24,
 'num_device_wires': 5,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

## Transpile to trapped ion gates

In [9]:
@qml.qnode(dev)
@single_qubit_fusion_gpi
@ionizer.transforms.commute_through_ms_gates(direction="right")
@single_qubit_fusion_gpi
@ionizer.transforms.commute_through_ms_gates(direction="right")
@ionize
def tapered_circuit_ionized(params):
    new_qfunc()
    qml.RZ(np.pi/4, wires=0)
    qml.RZ(np.pi/4, wires=1)
    qml.RZ(np.pi/4, wires=2)
    qml.RZ(np.pi/4, wires=3)
    qml.RZ(3 * np.pi/4, wires=4)
    return qml.probs()

In [10]:
print(qml.draw(tapered_circuit_ionized)(params))

0: ──GPI2(0.00)───GPI(-2.57)──GPI2(-0.79)─╭MS──GPI(-0.28)───────────────────────────╭MS──GPI2(0.07)
1: ──GPI2(-2.36)──GPI(0.39)───GPI2(1.57)──╰MS──GPI2(-1.57)──GPI(-0.50)──GPI2(-1.00)─╰MS─╭MS────────
2: ──GPI2(-2.36)──GPI(0.39)───GPI2(1.57)────────────────────────────────────────────────╰MS────────
3: ──GPI2(-2.36)──GPI(0.39)───GPI2(1.57)───────────────────────────────────────────────────────────
4: ──GPI2(-2.36)──GPI(0.39)───GPI2(1.57)───────────────────────────────────────────────────────────

───GPI(1.18)────GPI2(-0.86)────────────────────────────────────────────────────────────────────────
───GPI(-0.04)────────────────────────────╭MS──GPI2(0.07)──GPI(1.18)────GPI2(-0.86)─────────────────
───GPI2(-1.57)──GPI(2.40)────GPI2(-1.49)─╰MS─╭MS──────────GPI(-0.02)────────────────────────────╭MS
─────────────────────────────────────────────╰MS──────────GPI2(-1.57)──GPI(2.37)────GPI2(-1.54)─╰MS
───────────────────────────────────────────────────────────────────────────────────────────────────

In [11]:
qml.specs(tapered_circuit_ionized)(params)

{'resources': Resources(num_wires=5, num_gates=54, gate_types=defaultdict(<class 'int'>, {'GPI2': 28, 'GPI': 18, 'MS': 8}), gate_sizes=defaultdict(<class 'int'>, {1: 46, 2: 8}), depth=26, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 46, 2: 8}),
 'gate_types': defaultdict(int, {'GPI2': 28, 'GPI': 18, 'MS': 8}),
 'num_operations': 54,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 5,
 'num_trainable_params': 0,
 'depth': 26,
 'num_device_wires': 5,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}